<a href="https://colab.research.google.com/github/LuziVGC/detector-de-eclipses/blob/main/detector_de_eclipses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rebound

In [ ]:
#Librerias a utilizar
import rebound
import matplotlib.pyplot as plt
import numpy as np
import datetime
from datetime import date

In [ ]:
#Condiciones temporales
def convert_to_rebound_date(year, month, day):
  return(int(year) + (int(month-1))/12 + (int(day-1))/365.25)

fechai = input("Bienvenido/a al detector de eclipses solares, escoja una fecha desde la cual quiera detectar en el formato Año-Mes-Día: " )
print("")
fechaf = input("Y además hasta que fecha quiere simular en el formato Año-Mes-Día: ")

principio = fechai.split("-")
final = fechaf.split("-")

fecha1 = datetime.date(int(principio[0]), int(principio[1]), int(principio[2]))
fecha2 = datetime.date(int(final[0]), int(final[1]), int(final[2]))
diferencia = fecha2 - fecha1
dias = diferencia.days
days = int(diferencia.days)

hoy = datetime.datetime.now()
fechah = [hoy.year, hoy.month, hoy.day]

In [ ]:
sim = rebound.Simulation()
sim.units = ('Msun', 'AU', 'yr')

sim.add("Sun", date = hoy)
sim.add("399", date = hoy)
sim.add("301", date = hoy)
sim.move_to_com()
sim.status()

In [86]:
Noutputs = round((10000*days)/365.25)
today = convert_to_rebound_date(int(fechah[0]),int(fechah[1]),int(fechah[2]))
finish = convert_to_rebound_date(int(final[0]), int(final[1]), int(final[2]))
start = convert_to_rebound_date(int(principio[0]), int(principio[1]), int(principio[2]))
times = np.linspace((start-today),(finish - today),Noutputs)

x = np.zeros((3,Noutputs))
y = np.zeros((3,Noutputs))
z = np.zeros((3,Noutputs))

sol = sim.particles[0]
tie = sim.particles[1]
lun = sim.particles[2]

for i,t in enumerate(times):
  sim.integrate(t)
  x[0,i] = sol.x
  y[0,i] = sol.y
  z[0,i] = sol.z
  x[1,i] = tie.x
  y[1,i] = tie.y
  z[1,i] = tie.z
  x[2,i] = lun.x
  y[2,i] = lun.y
  z[2,i] = lun.z


In [ ]:
def area_of_triangle(v1, v2):
    return np.linalg.norm(np.cross(v1, v2))

tl = []
ts = []
sl = []
tlx = x[2]-x[1]
tly = y[2]-y[1]
tlz = z[2]-z[1]
tsx = x[1]-x[0]
tsy = y[1]-y[0]
tsz = z[1]-z[0]
slx = x[2]-x[0]
sly = y[2]-y[0]
slz = z[2]-z[0]
tl.append(tlx)
tl.append(tly)
tl.append(tlz)
ts.append(tsx)
ts.append(tsy)
ts.append(tsz)
sl.append(tsx)
sl.append(tsy)
sl.append(tsz)

a = 0
eclipses = []
areas = []
while a < Noutputs:
  v1 = np.array([tl[0][a],tl[1][a],tl[2][a]],float)
  v2 = np.array([ts[0][a],ts[1][a],ts[2][a]],float)
  v3 = np.array([sl[0][a],sl[1][a],sl[2][a]],float)
  magnitud = area_of_triangle(v1,v2)
  if magnitud < 0.00003:
    eclipses.append(times[a])
  areas.append(magnitud)
  a += 1

eclipse_dates = [hoy + datetime.timedelta(days = t*365.25) for t in eclipses]
print("Fechas de eclipse:")
for date in eclipse_dates:
    print(date.strftime("%Y-%m-%d"))

In [ ]:
plt.plot(times, areas)
plt.xlabel('Fecha (años)')
plt.ylabel('Área del triángulo (UA$^2$)')
plt.title('Variación del área del triángulo formado por el Sol, la Tierra y la Luna')
plt.xticks(rotation=45)
plt.grid(True)

for eclipse_time in eclipses:
    plt.axvline(x=eclipse_time, color='r', linestyle='--', linewidth=1)

plt.tight_layout()
plt.show()